# Analyzing the Effects of Fintech on the Banking Industry with Foursquare Developers API
### Ian Zelaya
***

## Introduction 

This project will provide a good exploratory analysis on how fintech has affected different regions around the world. It will help entrepreneurs understand what cities or regions could be optimal for fintech entrepreneurship and investors to find locations with great investment opportunities in the fintech industry.
<br></br>
<br></br>
In the last couple of years, the fintech industry has revolutionized how the banking industry interacts with its customers. This trend began with movements such as the PayPal company, where transfers of money are made via online. Now, more and more traditional banking services are being transferred to a digital form. Some countries, such as Singapore, have further deepened the usage of fintech to the point where most transactions, either for groceries or clothing, are made through digital devices. In the past, banks required to invest a great portion of their money into infrastructure to maintain clients from different areas of a city. Now that fintech services are available, many people prefer to adapt to the technology rather than wasting time in line. This suggests that banks are now able to reduce their investments on infrastructure, gain their customers via online, and focus on more fintech. If this argument is true, there should be evidence of a decreased amount of bank establishments. 
<br></br>
<br></br>
The objective of this project, besides understanding how fintech has affected the banking industry, is to use data in the 
Foursquare API. From the API, we will access information of all banking establishments in different cities. The best approach to evidence if fintech has significantly reduced the amount of bank establishments is through time series data. In this approach, the data would be easier to relate it with fintech innovation. Unfortunately, Foursquare API lacks time series data. As an alternative, we will try to evaluate three different hypotheses: 

- Divergence between countries with high and low fintech innovation 
- Divergence between big and small cities
- Divergence between neighborhoods of high and low income 

The first hypothesis assumes that countries which are more developed in fintech will have lower levels of bank establishments than countries with lower fintech. The second hypothesis assumes a divergence between big and small cities, as in theory, big cities should become early adopters of fintech. At last, the third hypothesis assumes a divergence between high versus low income neighborhoods for the same reason as hypothesis two.

### Data
More specifically, this project will focus on finding divergence in the amount of bank establishments per population of a city or neighborhood. Our hypotheses will be measured in the following manner:

- Part I, Geographic difference: Bank establishments per city population 
- Part II, Big and small cities: Bank establishments per city population 
- Part III, High and low income: Bank establishments per neighborhood population 

Foursquare API requires of a location input in order to return a list of venues close to that location. Fortunately, Openmapstreet API provides all the location data required to run Foursquare API. For part II and part III, we will also need to find data related to the population and income of cities and neighborhoods. After some research, I found out that Statistical Atlas displays this information through its website. The data they use is extracted from the United States Census Bureau, which is a reliable source. Web scraping will be necessary to extract the information from the Statistical Atlas Website.

## Importing Libraries and Cloning Repositories

In [814]:
# Import the necessary libraries for the project
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim 
import geocoder
from sklearn.cluster import KMeans
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import json 
import geojson
from geojson import Feature, FeatureCollection, dump
import shapely.wkt
from shapely.geometry import Point
from IPython.display import Image

## Countries with High and Low FinTech Innovation

The first analysis will be the divergence between countries of high and low fintech innovation. We will use the capital city as a representation for the countries to analyze. There are 195 countries in the world. We could implement an analysis which considers every country of the world but that would imply a deeper data mining as some developing countries have insufficient or hard to access data. For that reason, it is necessary to find a way to sample countries which have high fintech and low fintech. The Institute for Financial Services Zug (IFZ) of the Lucerne University of Applied Sciences has conducted a reliable and comprehensive [research](https://blog.hslu.ch/retailbanking/files/2019/03/IFZ-FinTech-Study-2019_Switzerland.pdf) to identify the regions which have the highest levels of fintech. They evaluated the political, economic, technological and social factors in order to create a fintech ranking. In the top of the ranking, Singapore, Zurich and Amsterdam were rated the highest. From the 33 cities analyzed, Buenos Aires, Mumbai and Sao Paulo were rated the lowest. For this project, we will use the three highest rated countries as samples for the high fintech countries and the bottom three as samples for the low fintech countries. 

Now that we have defined the cities to analyze, we must determine the way to extract the amount of banks establishments per city. To execute this task, we will make usage of GeoJSON data. A GeoJSON file is a format for encoding a variety of geographic data structures. In our case, we can create GeoJSON files that represent the boundaries of a city or neighborhood through data structures such as Polygons and Multipolygons. Fortunately for us, Openstreetmap API has already constructed these Polygons and Multipolygons for cities and neighborhoods. We will extract this data through the Geocoder library as it has an easy implementation to manipulate the Openstreetmap API data. The following diagram illustrates the data flow process and will detail the steps in the next paragraph. Here is the diagram: 

![Data Flow Diagram](Fintech_diagram.png)

First, we begin by extracting the city's polygon through the Openstreetmap API. Using the polygon, we will create random points inside the city. The reason for creating these random points is that Foursquare API has a limit of 100 venues to return for any location given. So, for example, if we analyze a radius of 5KM around a location and there are more than 100 venues in that radius, we won't be able to extract all the venues. By generating random points inside the polygon of a city, we will be able to extract all the banks venues within. The Shapely library has a function named "bounds" which serves perfectly to implement this random data. Basically, the bounds function creates random points within maximum X and Y values. In this case, the X and Y values are the maximum and minimum latitude and longitude of a polygon. Finally, we are ready to input the random points into Foursquare API and return the bank venues. Note that many of the random points will return repeated bank establishments, but this won't be a problem as a simple filtering of duplicated longitude and latitude will solve the issue. We will also be able to generate a geographic map with Folium Library to visualize the random points or bank venues inside the polygon. 

The analyses of this project can be considered as an iterative analysis for each city or neighborhood. As a result, the generation of functions to automate processes will become really useful. The function "random_point_generator" executes the process from the data flow diagram up to the Shapely random points generation. The function requires as input a name or list of names of cities and has the option to specify the number of random points to generate. It also accepts data in a neighborhood format, but that will be explained in part III of this project. As output, the function  generates a Python dictionary with keys for each city specified in the list. At the same time, "random_point_generator" saves the Folium map and the Polygon GeoJSON file in case the Openstreetmap API returns an error (which is common) while executing the for loop.

In [604]:
# Obtain a polygon representing the borders of the cities and create random locations inside the polygon to sample 
# the whole city

import random 

list_of_cities1 = ['Buenos Aires, Argentina', 'Mumbai, India', 'Amsterdam, Netherlands', 'Zurich, Switzerland', 
                   'Sao Paulo, Brazil', 'Singapore']

list_of_map1 = ['Baires', 'Mumbai', 'Amsterdam', 'Zurich', 'Sao Paulo', 'Singapore']


def random_point_generator(list_of_cities, list_of_map, num_points = 75, neigh_format = False, list_of_neigh = False):
    dict_maps = dict()
    for i in range(len(list_of_cities)):

        # Create an API request from the Openstreetmap API in order to extract the location of the cities and 
        # its borders coordinates.
        geolocator2 = Nominatim(user_agent='ny_explorer4')
        
        if neigh_format == False:
            geometry_wkt = geolocator2.geocode('{}'.format(list_of_cities[i]), geometry='wkt')
            
            if geometry_wkt == None:
                pass
            
            else:

                loc = geolocator2.geocode(geometry_wkt)
                lat_trial  = geometry_wkt.latitude
                lng_trial = geometry_wkt.longitude

                # obtain the locations that compose the borders of the city 
                geometry = geometry_wkt.raw['geotext']

                # Use location of the city to center the map 
                m = folium.Map(location=[lat_trial, lng_trial], zoom_start=12)

                # Transform the border locations into a Polygon through the Shapely library 
                P = shapely.wkt.loads(geometry)
                features = Feature(geometry=P, properties={'name': '{}'.format(list_of_cities[i])})

                # Save the polygon as a GeoJson file 
                with open('testgeo_{}.geojson'.format(list_of_cities[i]), 'w') as f:
                    dump(features, f)

                # Load the polygon for further analysis 
                with open('testgeo_{}.geojson'.format(list_of_cities[i]), 'r') as f:
                    gj = geojson.load(f)

                # The bounds function from the Shapely library returns the maximum Y and X values from a Polygon. These values will be
                # used to calculate locations inside the polygon that ranges within these maximums and minimums. 
                env = P.bounds
                xmin, ymin, xmax, ymax = env[0],env[1],env[2],env[3]
                num_points = num_points
                counter = 0

                # Create the random locations within the cities' polygon 
                l = []
                for j in range(num_points):
                    while counter < num_points:

                        point = Point(random.uniform(xmin, xmax),
                                       random.uniform(ymin, ymax))

                        if point.within(P):
                            l.append(point)
                            counter += 1

                # Switch the latitude and longitude values as the Nominatim API returns the locations as "[Lng, Lat]" format instead of 
                # the standard "[lat, lng]" format. 
                list_arrays = [ np.array((geom.xy[1][0], geom.xy[0][0])) for geom in l ]
                dict_maps['{}'.format(list_of_cities[i])] = list_arrays

                # Add the cities' polygon into the folium map 
                folium.Choropleth(
                      geo_data='testgeo_{}.geojson'.format(list_of_cities[i]),
                      name='choropleth',
                      fill_opacity=0,
                      line_opacity=0.8,
                      line_color = 'blue').add_to(m)

                # Add the random location points into the folium map
                for point in list_arrays:
                    folium.CircleMarker(
                    point,
                    radius=5,
                    fill=True,
                    fill_opacity=0.7).add_to(m)

                print('Done: {}'.format(list_of_cities[i]))

                # save the folium map containing the polygon and points to print it later
                m.save('map_{}.html'.format(list_of_map[i]))
            
        else:
            geometry_wkt = geolocator2.geocode('{}, {}'.format(list_of_neigh[i], list_of_cities[i]), geometry='wkt')
            
            if geometry_wkt == None:
                pass
            
            else:
       
                loc = geolocator2.geocode(geometry_wkt)
                lat_trial  = geometry_wkt.latitude
                lng_trial = geometry_wkt.longitude

                # obtain the locations that compose the borders of the city 
                geometry = geometry_wkt.raw['geotext']

                # Use location of the city to center the map 
                m = folium.Map(location=[lat_trial, lng_trial], zoom_start=12)

                # Transform the border locations into a Polygon through the Shapely library 
                P = shapely.wkt.loads(geometry)
                features = Feature(geometry=P, properties={'name': '{}'.format(list_of_cities[i])})

                # Save the polygon as a GeoJson file 
                with open('testgeo_{}.geojson'.format(list_of_neigh[i]), 'w') as f:
                    dump(features, f)

                # Load the polygon for further analysis 
                with open('testgeo_{}.geojson'.format(list_of_neigh[i]), 'r') as f:
                    gj = geojson.load(f)

                # The bounds function from the Shapely library returns the maximum Y and X values from a Polygon. These values will be
                # used to calculate locations inside the polygon that ranges within these maximums and minimums. 
                env = P.bounds
                xmin, ymin, xmax, ymax = env[0],env[1],env[2],env[3]
                num_points = num_points
                counter = 0

                # Create the random locations within the cities' polygon 
                l = []
                for j in range(num_points):
                    while counter < num_points:

                        point = Point(random.uniform(xmin, xmax),
                                       random.uniform(ymin, ymax))

                        if point.within(P):
                            l.append(point)
                            counter += 1

                # Switch the latitude and longitude values as the Nominatim API returns the locations as "[Lng, Lat]" format instead of 
                # the standard "[lat, lng]" format. 
                list_arrays = [ np.array((geom.xy[1][0], geom.xy[0][0])) for geom in l ]
                dict_maps['{}'.format(list_of_neigh[i])] = list_arrays

                # Add the cities' polygon into the folium map 
                folium.Choropleth(
                      geo_data='testgeo_{}.geojson'.format(list_of_neigh[i]),
                      name='choropleth',
                      fill_opacity=0,
                      line_opacity=0.8,
                      line_color = 'blue').add_to(m)

                # Add the random location points into the folium map
                for point in list_arrays:
                    folium.CircleMarker(
                    point,
                    radius=5,
                    fill=True,
                    fill_opacity=0.7).add_to(m)

                print('Done: {}'.format(list_of_neigh[i]))

                # save the folium map containing the polygon and points to print it later
                m.save('map_{}.html'.format(list_of_map[i]))

        
    return dict_maps

dict_maps1 = random_point_generator(list_of_cities1, list_of_map1)

Once we have the Python dictionary with the random points for each city, we continue the process by using the random points to extract the bank venues. However, before continuing the analysis, it is a good idea to save Python dictionary if we ever want to use it for further analysis. The function "save_dict_maps" saves a dictionary to the local file in a pickle format and declares a variable with the dictionary for immediate usage. The pickle format is a Python-specifit data format and is one of the highest and fastest compression formats to use in Python. The disadvantage, however, is that the pickle format is not available for usage in other programming languages. In this case, the pickle format is the appropiate option to save data files as we will use it only internally.

In [216]:
import pickle

def save_dict_maps(filename, dict_m):
    with open('{}.pickle'.format(filename), 'wb') as handle:
        pickle.dump(dict_m, handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open('{}.pickle'.format(filename), 'rb') as handle:
        dict_maps = pickle.load(handle)
        
    return dict_maps
    
dict_maps1 = save_dict_maps('dict_maps1', dict_maps1)
print(dict_maps1.keys())

dict_keys(['Buenos Aires, Argentina', 'Mumbai, India', 'Amsterdam, Netherlands', 'Zurich, Switzerland', 'Sao Paulo, Brazil', 'Singapore'])


For verification purposes, we will display the maps generated for each city with their respective random data points. Here we can see the effectiveness of using Shapely's "bounds" function as every data point is generated inside the polygon. As seen in the pictures below, the random points cover the area inside the polygon.

In [87]:
from IPython.display import IFrame

print('Buenos Aires:')
display(IFrame(src='map_Baires.html', width=900, height=600))

print('Mumbai:')
display(IFrame(src='map_Mumbai.html', width=900, height=600))

print('Amsterdam:')
display(IFrame(src='map_Amsterdam.html', width=900, height=600))

print('Zurich:')
display(IFrame(src='map_Zurich.html', width=900, height=600))

print('Sao Paulo:')
display(IFrame(src='map_Sao Paulo.html', width=900, height=600))

print('Singapore:')
display(IFrame(src='map_Singapore.html', width=900, height=600))

Buenos Aires:


Mumbai:


Amsterdam:


Zurich:


Sao Paulo:


Singapore:


Next, the "venue_extractor" function will convert the random points into the desired bank venues. To implement this process, the function applies a nested for loop to apply the analysis to each data point from each city. Inside the nested for loop, the function executes a get request from the Foursquare API to extract all bank venues within a radius of 5KM from the specific data point. The API responds by sharing the bank name, location and more features related to every bank venue. We will only extract the bank name, latitude and longitude to later eliminate duplicated bank venues by comparing repeated latitude and longitude values. 

An interesting library function used for "venue_extractor" was the "from_iterable" function from the itertools library. With the "from_iterable" function, we are able to convert a group of Python lists into a single list. In our case, we use this function to group the list of bank venues for each city. Similar to the function "random_point_generator", this function also accepts dictionaries for neighborhood analysis, but that will later be explained in part III.

In [737]:
import itertools

print(dict_maps1.keys())
        

def venue_extractor(dict_m, radius = 5000, neigh_format=False, neigh_data=None):
    CLIENT_ID = 'VEX253QKRHQKOQAINBVZFMPFR4KQF01VH035ARG3TQ0TPD5L'
    CLIENT_SECRET = 'G3EIDS4K12GQ32ELNIL1BIQAJOBP440MEP3RQJHTVUGN0B34'
    VERSION = '20200828'

    
    if neigh_format == False: 

        cities_venues = []
        
        for city in dict_m.keys():
            print(city)
            city_venues = []

            for loc in dict_m[city]:
                lat = loc[0]
                lng = loc[1]

                url_try = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
                        CLIENT_ID, 
                        CLIENT_SECRET, 
                        VERSION, 
                        lat, 
                        lng, 
                        '4bf58dd8d48988d10a951735',
                        radius,
                        100)

                results_try = requests.get(url_try).json()["response"]['groups'][0]['items']

                city_venues.append([(
                    v['venue']['name'],
                    v['venue']['location']['lat'], 
                    v['venue']['location']['lng']) for v in results_try])

            city_venue_full = list(itertools.chain.from_iterable(city_venues))
            cities_venues.append(city_venue_full)
            
        return cities_venues
            
    else:
        
        neighborhoods_venues = []
        
        for neigh in dict_m.keys():
            print(neigh)
            neigh_venues = []
            neigh_query = ngh_cities[ngh_cities['Neighborhood'].str.contains(neigh)].values.tolist()[0]

            for loc in dict_m[neigh]:
                lat = loc[0]
                lng = loc[1]

                url_try = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
                        CLIENT_ID, 
                        CLIENT_SECRET, 
                        VERSION, 
                        lat, 
                        lng, 
                        '4bf58dd8d48988d10a951735',
                        radius,
                        100)

                results_try = requests.get(url_try).json()["response"]['groups'][0]['items']

                neigh_venues.append([(
                    neigh_query[0],
                    neigh,
                    neigh_query[2],
                    neigh_query[3],
                    v['venue']['name'],
                    v['venue']['location']['lat'], 
                    v['venue']['location']['lng']) for v in results_try])


            neigh_venue_full = list(itertools.chain.from_iterable(neigh_venues))
            neighborhoods_venues.append(neigh_venue_full)
        
        return neighborhoods_venues
            



#cities_venues1 = venue_extractor(dict_maps1)

dict_keys(['Buenos Aires, Argentina', 'Mumbai, India', 'Amsterdam, Netherlands', 'Zurich, Switzerland', 'Sao Paulo, Brazil', 'Singapore'])


In [227]:
with open('cities_venues.pickle', 'wb') as handle:
     pickle.dump(cities_venues, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('cities_venues.pickle', 'rb') as handle:
     cities_venues = pickle.load(handle)
        
cities_venues1 = save_dict_maps('cities_venues1', cities_venues1)

# Buenos Aires bank establishments in dataframe format
baires_df = pd.DataFrame(cities_venues1[0], columns=['Bank Establishments', 'Lat', 'Long'])
print('Buenos Aires: ')
display(baires_df.head())
print(' ')

# Mumbai bank establishments in dataframe format
mumbai_df = pd.DataFrame(cities_venues1[1], columns=['Bank Establishments', 'Lat', 'Long'])
print('Mumbai: ')
display(mumbai_df.head())
print(' ')

# Amsterdam bank establishments in dataframe format
amsterdam_df = pd.DataFrame(cities_venues1[2], columns=['Bank Establishments', 'Lat', 'Long'])
print('Amsterdam: ')
display(amsterdam_df.head())
print(' ')

#Zurich bank establishments in dataframe format 
zurich_df = pd.DataFrame(cities_venues1[3], columns=['Bank Establishments', 'Lat', 'Long'])
print('Zurich: ')
display(zurich_df.head())
print(' ')

#Sao Paulo bank establishments in dataframe format
spaulo_df = pd.DataFrame(cities_venues1[4], columns=['Bank Establishments', 'Lat', 'Long'])
print('Sao Paulo: ')
display(spaulo_df.head())
print(' ')

#Singapore bank establishments in dataframe format
sng_df = pd.DataFrame(cities_venues1[5], columns=['Bank Establishments', 'Lat', 'Long'])
print('Singapore: ')
display(sng_df.head())
print(' ')

Buenos Aires: 


,Bank Establishments,Lat,Long
0,Santander Río,-34.628997,-58.462739
1,BBVA Francés,-34.627533,-58.458409
2,Santander,-34.631442,-58.454573
3,Banco Galicia,-34.629910,-58.465902
4,Banco Nación,-34.625676,-58.458197


 
Mumbai: 


,Bank Establishments,Lat,Long
0,Kotak Mahindra Bank - ATM,19.031531,72.841719
1,Kotak Mahindra Bank - Branch/ATM,19.035435,72.842348
2,Kotak Mahindra Bank - Branch/ATM,19.026622,72.853625
3,Kotak Mahindra Bank - ATM,19.015915,72.843231
4,Kotak Mahindra Bank - ATM,19.040349,72.862339


 
Amsterdam: 


...

,Bank Establishments,Lat,Long
0,Bank Vontobel AG,47.365032,8.534345
1,Zurich Insurance Group,47.362044,8.533978
2,UBS,47.369683,8.538556
3,Credit Suisse,47.370594,8.538784
4,UBS,47.371875,8.538559


 
Sao Paulo: 


,Bank Establishments,Lat,Long
0,Itaú,-23.611098,-46.600269
1,CRK Informática LTDA,-23.614724,-46.569143
2,Bradesco,-23.625624,-46.580339
3,Itaú Personnalité,-23.616190,-46.571776
4,Itaú,-23.630652,-46.603876


 
Singapore: 


,Bank Establishments,Lat,Long
0,Maybank,1.378551,103.736771
1,Ocbc Bank @ Jurong West,1.351605,103.712935
2,DBS,1.384593,103.743317
3,OCBC,1.350818,103.718532
4,OCBC Bank,1.350663,103.718585


We now have the bank establishments from all cities and converted them into dataframes for each city. However, there is duplicated data as many of the random points generated have common bank venues in their radius. To solve this problem, we use the "drop_duplicated" function from Pandas. We will then have a dataframe with unique bank establishments and now we are able to count the number of establishments with the "len" built-in function of Python. Then, we construct a new dataframe with the number of banks venues, population and banks per million of population. Here we sort values with pandas and analyze the results. 

As we can in the final dataframe, by excluding Mumbai and Zurich, there seems to be a difference in bank venues for cities of high and low fintech innovation. Sao Paulo and Buenos Aires averaged a ratio of 134 while Singapore and Amsterdam averaged a ratio of 111. The results are tempting at most, as we would need further analysis to make statistical conclusions. The reason for such a low ratio from Mumbai could be their overpopulation and, for Zurich, the reason for a high ratio could be their small but concentrated area as a financial center. 

In [106]:
baires_df = baires_df.drop_duplicates()

mumbai_df = mumbai_df.drop_duplicates()

amsterdam_df = amsterdam_df.drop_duplicates()

zurich_df = zurich_df.drop_duplicates()

spaulo_df = spaulo_df.drop_duplicates()

sng_df = sng_df.drop_duplicates()

list_of_df = [baires_df, mumbai_df, amsterdam_df, zurich_df, spaulo_df, sng_df]
list_pop = [2.890, 18.400, 0.821, 0.402762, 12.180, 5.850]

list_for_display = []
for dfs in range(len(list_of_df)):
    l = len(list_of_df[dfs])
    ratio = l/list_pop[dfs]
    list_for_display.append([list_of_cities[dfs], l, list_pop[dfs], ratio])
    
df_whole = pd.DataFrame(list_for_display, columns=['Cities', 'Banks', 'Population', 'Banks/Populatation (per Million)'])
df_whole = df_whole.reset_index(drop=True)
df_whole = df_whole.set_index('Cities')
display(df_whole.sort_values(by=['Banks/Populatation (per Million)']))

,Banks,Population,Banks/Populatation (per Million)
Cities,,,
"Mumbai, India",754,18.400000,40.978261
"Amsterdam, Netherlands",89,0.821000,108.404385
Singapore,665,5.850000,113.675214
"Buenos Aires, Argentina",382,2.890000,132.179931
"Sao Paulo, Brazil",1654,12.180000,135.796388
"Zurich, Switzerland",68,0.402762,168.834200


## Big and Small Cities Analysis

In part II, we will analyze the difference of bank venues in big versus small cities. To continue with the analysis, we must first determine an appropiate filter to extract a sample of cities which will represent both groups. I decided to implement this analysis by controlling the country factor. In other words, I will select cities from the same country to avoid any data disturbance from differences between countries. The United States is an attractive option due to the extensive public data of the country and at the same time has an ample category of cities. Now that we selected the country to analyze, we will proceed to start part II.

To begin, we need a list of cities within the U.S. and extract the biggest and smallest cities. To extract this list, we will use the list of cities of the U.S. published at Wikipedia, and their respective population. The library BeautifulSoup is a Python package for parsing HTML documents and will be of importance for our data extraction. With BeautifulSoup, we are able to extract sections, subsections or items from the HTML document. In our case, we want to extract the table section from the website which containts the list of cities, their population and more related data. Once we have the table defined within a variable in Python, we can further analyze the table to find items with particular characteristics. After reading the HTML code in the website (right-click and inspect option with Google Chrome), we can find the code that differenciates the html elements we want to extract from any other html elements. For the cities name data, this code is a hyperlink attribute and for the population data, the unique code is the style attribute. Having the unique attributes for each data feature, we are now able to apply a for loop in the table and extract the city name and population for each row (or each city). After converting all the data into a dataframe, we order the list of cities by the population column and extract the top 3 and bottom 3. As we can see in the modified dataframe, the top 3 cities are New York, Los Angeles and Chicago. For the bottom 3, we obtained Clinton, Bend and Woodbridge.

In [816]:
# Parse the website that contains the list of cities in the U.S. and their respective populations
url_2 = 'https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population'
url_req_2 = requests.get(url_2)
soup_2 = BeautifulSoup(url_req_2.content, 'html.parser')
table_2 = soup_2.findAll('table')[4]
table_rows_3 = table_2.findAll('tr')

# We will use rows_2 to save city name and popultion for each city
rows_2 = []

# For loop to extract the city name and population from the table 
for rs in table_rows_3:
    rows = []
    
    # counters useful to extract only the cells that we want from the table
    counter_h = []
    counter_p = []

    # extract all hyperlinks from each row. Within these hyperlinks, we will extract the text which have the city names
    hype = rs.findAll('a')
    
    # extract cells in the table which have the style used only for numeric cells such as population
    pop = rs.findAll('td', {'style': 'text-align:right;'})

    # for loop to find the name of cities 
    for h in hype:
        counter_h.append(1)
        # Pass hyperlinks "a" which are not useful for our analysis. For example, '[d]' is a link to the notes section
        # in Wikipedia. 
        if h.text[0] == '[':
            pass
        
        # Append the first cell exclusively as we only want the name of the city and ignore other data in the table
        elif len(counter_h) == 1:
            rows.append(h.text)
            
        else: 
            pass
    # for loop to find the population of cities
    for p in pop:
        counter_p.append(1)
        
        # In the same way to find the city names, we will append only the first cell and ignore other data in the table
        if len(counter_p) == 1:
            rows.append(p.text[:-1])
            
        else:
            pass
        
    rows_2.append(rows)

    
# Convert the rows_2 into a dataframe
df_cities2 = pd.DataFrame(rows_2[1:], columns=['City', 'Population'])
df_cities2['Population'] = df_cities2['Population'].str.replace(',', '')
df_cities2["Population"] = pd.to_numeric(df_cities2["Population"], downcast="float")


# Extract the 3 cities with the highest and lowest populations
df_cities2 = pd.concat([df_cities2.head(3), df_cities2.tail(3)])
df_cities2 = df_cities2.reset_index(drop=True)
display(df_cities2)

,City,Population
0,New York,8336817.0
1,Los Angeles,3979576.0
2,Chicago,2693976.0
3,Clinton,100471.0
4,Bend,100421.0
5,Woodbridge,100145.0


Now we will use the list of cities as input for the "random_point_generator" and save the random points generated for each city. In the same way as part I, we visualize the random points to confirm that they are within the polygon.

In [226]:
list_of_map2 = ['newyork', 'LA', 'Chicago', 'Clinton', 'Bend', 'WoodB']

dict_maps2 = random_point_generator(df_cities2['City'], list_of_map2)

dict_maps2 = save_dict_maps('dict_maps2', dict_maps2)
print(dict_maps2.keys())

Done: New York
Done: Los Angeles
Done: Chicago
Done: Clinton
Done: Bend
Done: Woodbridge
dict_keys(['New York', 'Los Angeles', 'Chicago', 'Clinton', 'Bend', 'Woodbridge'])


In [229]:
print('New York:')
display(IFrame(src='map_newyork.html', width=900, height=600))

print('Los Angeles:')
display(IFrame(src='map_LA.html', width=900, height=600))

print('Chicago:')
display(IFrame(src='map_Chicago.html', width=900, height=600))

print('Clinton:')
display(IFrame(src='map_Clinton.html', width=900, height=600))

print('Bend:')
display(IFrame(src='map_Bend.html', width=900, height=600))

print('Woodbridge:')
display(IFrame(src='map_WoodB.html', width=900, height=600))

New York:


Los Angeles:


Chicago:


Clinton:


Bend:


Woodbridge:


Then, we will use the dictionary of random points for the "venue_extractor" function and extract the bank venues for each city. 

In [265]:
cities_venues2 = venue_extractor(dict_maps2)

cities_venues2 = save_dict_maps('cities_venues2', cities_venues2)

New York
Los Angeles
Chicago
Clinton
Bend
Woodbridge


In [269]:
ny_df = pd.DataFrame(cities_venues2[0], columns=['Bank Establishments', 'Lat', 'Long'])
print('New York: ')
display(ny_df.head())

la_df = pd.DataFrame(cities_venues2[1], columns=['Bank Establishments', 'Lat', 'Long'])
print('Los Angeles: ')
display(la_df.head())

chicago_df = pd.DataFrame(cities_venues2[2], columns=['Bank Establishments', 'Lat', 'Long'])
print('Chicago: ')
display(chicago_df.head())

clinton_df = pd.DataFrame(cities_venues2[3], columns=['Bank Establishments', 'Lat', 'Long'])
print('Clinton: ')
display(clinton_df.head())

bend_df = pd.DataFrame(cities_venues2[4], columns=['Bank Establishments', 'Lat', 'Long'])
print('Bend: ')
display(bend_df.head())

woodb_df = pd.DataFrame(cities_venues2[5], columns=['Bank Establishments', 'Lat', 'Long'])
print('Woodbridge: ')
display(woodb_df.head())

New York: 


,Bank Establishments,Lat,Long
0,Chase Bank,40.580728,-73.838091
1,Citibank,40.581998,-73.835019
2,HSBC Bank,40.581548,-73.838567
3,Coinstar,40.582800,-73.834700
4,"Queens County Savings Bank, a division of New ...",40.583558,-73.825574


Los Angeles: 


,Bank Establishments,Lat,Long
0,Chase Bank,33.723117,-118.313580
1,Bank of America,33.723464,-118.312617
2,Bank of America,33.735302,-118.288231
3,Coinstar,33.723923,-118.313446
4,Malaga Bank,33.722331,-118.310912


Chicago: 


...

,Bank Establishments,Lat,Long
0,Tempo Bank,38.605249,-89.683336
1,Community Bank of Trenton,38.605290,-89.681070
2,Germantown trust and savings bank,38.609538,-89.535787
3,First Bank,38.608596,-89.528468
4,Coinstar,38.619736,-89.373703


Bend: 


,Bank Establishments,Lat,Long
0,U.S. Bank Branch,44.038345,-121.304596
1,Chase Bank,44.034520,-121.309087
2,First Interstate Bank,44.024077,-121.314390
3,Coinstar,44.033100,-121.308400
4,Home Federal Bank,44.028514,-121.313274


Woodbridge: 


,Bank Establishments,Lat,Long
0,Wells Fargo,40.548751,-74.294735
1,Bank of America,40.553293,-74.281743
2,TD Bank,40.570111,-74.290856
3,The Provident Bank,40.551748,-74.281250
4,CJFCU,40.555166,-74.275192


In [283]:
list_of_df2 = [ny_df, la_df, chicago_df, clinton_df, bend_df, woodb_df]


list_for_display2 = []
for dfs in range(len(list_of_df2)):
    # Drop duplicates 
    non_duplc = list_of_df2[dfs].drop_duplicates()
    l = len(non_duplc)
    
    # Calculate ratio of bank establishments per million of population
    ratio = l/(df_cities2['Population'][dfs]/1000000)
    list_for_display2.append([df_cities2['City'][dfs], l, df_cities2['Population'][dfs], ratio])
    

df_whole2 = pd.DataFrame(list_for_display2, columns=['Cities', 'Banks', 'Population', 'Banks/Population (per Million)'])
df_whole2 = df_whole2.reset_index(drop=True)
df_whole2 = df_whole2.set_index('Cities')
display(df_whole2.sort_values(by=['Banks/Population (per Million)']))

1679
201.39580849621623
1308
328.67823104772975
1170
434.30231004285116
21
209.0155368215704
53
527.7780543910138
108
1078.4362674122522


,Banks,Population,Banks/Population (per Million)
Cities,,,
New York,1679,8336817.0,201.395808
Clinton,21,100471.0,209.015537
Los Angeles,1308,3979576.0,328.678231
Chicago,1170,2693976.0,434.302310
Bend,53,100421.0,527.778054
Woodbridge,108,100145.0,1078.436267


## Neighborhoods of High and Low Income

In this section, we will evaluate the differences between neighborhoods of high and low income. For this ocasion, we will use a research paper published by the Federal Reserve Bank of New York to decide what cities to analyze. The research paper focuses on analyzing the income gaps in different cities of the U.S. and unsurprisingly, cities with high levels of urbanization demonstrated the highest levels of income gap. In particular, New York, Washington, Chicago, Houston and Los Angeles were among the cities with the highest income gap.  

In [517]:
def city_neigh_extractor(state, city):
    
    # State must go with capital letters and words separated by a hyphen
    primary_link = 'https://statisticalatlas.com/place/{}/{}/Overview'.format(state, city)
    primary_req = requests.get(primary_link)

    soup = BeautifulSoup(primary_req.content, 'html.parser')

    # Extract the hyperlinks with the name of neighborhoods and convert it into text. The spaces, ' ', are converted into 
    # '-' so that we could later build the website for the specific neighborhood.
    hyps = soup.findAll('a')
    
    texter = []
    for h in hyps:
        if '/neighborhood/' in h.get('href'):
            texter.append((h.text).replace(' ', '-'))            

    neigh_data = []
    # Extract Population 
    for n in texter:
        secon_link = 'https://statisticalatlas.com/neighborhood/{}/{}/{}/Population'.format(state, city, n)
        secon_req = requests.get(secon_link)
        secon_str = str(secon_req)
        
        # Ignore if there is no website when openning the hyperlink to a particular neighborhood
        if secon_str == '<Response [404]>':
            pass
        
        else:
            soup2 = BeautifulSoup(secon_req.content, 'html.parser')

            table_pop = soup2.findAll('div', {'class': 'sidebar-info-table{}clearfix'.format(' ')})[0]
            tr_pop = (table_pop.findAll('td')[0].text).replace(',', '')

            # Extract Income
            third_link = 'https://statisticalatlas.com/neighborhood/{}/{}/{}/Household-Income'.format(state, city, n)
            third_req = requests.get(third_link)
            soup3 = BeautifulSoup(third_req.content, 'html.parser')

            print(n)
            if int(tr_pop) > 25:
                # Sometimes there is no data related to a neighborhood due to its small size. These neighborhoods should be 
                # ignored.
                try:
                    table_inc = soup3.find('div', {'class': 'figure-contents'})
                    tr_inc = table_inc.findAll('g')[15]
                    td_inc = ((tr_inc.find('title').text)[1:]).replace(',', '')

                    neigh_data.append([city, n, tr_pop, td_inc])
                except:
                    pass

    return neigh_data

neigh_ny_2 = pd.DataFrame(city_neigh_extractor('New-York', 'New-York'),
                          columns=['City', 'Neighborhood', 'Population', 'Household Income'])

Annadale
Arden-Heights
Arrochar
Arverne
Astoria
Astoria-Heights
Auburndale
Bath-Beach
Battery-Park
Bay-Ridge
Bay-Terrace
Bay-Terrace
Baychester
Bayside
Bedford-Park
Bedford-Stuyvesant
Belle-Harbor
Bellerose
Belmont
Bensonhurst
Bergen-Beach
Blissville
Bloomfield
Boerum-Hill
Borough-Park
Breezy-Point
Briarwood
Brighton-Beach
Broad-Channel
Bronx-Park
Brooklyn-Heights
Brownsville
Bulls-Head
Bushwick
Butler-Manor
Cambria-Heights
Canarsie
Carnegie-Hill
Carroll-Gardens
Castle-Hill
Castleton-Corners
Central-Park
Charleston
Chelsea
Chelsea-Travis
Chinatown
City-Island
Clason-Point
Clifton
Clinton
Clinton-Hill
Clove-Lake
Co-op-City
Cobble-Hill
College-Point
Columbia-Street-Waterfront-District
Columbus-Circle
Concourse
Coney-Island
Corona
Country-Club
Crown-Heights
DUMBO
Dongan-Hills
Douglaston-Little-Neck
Downtown
Dyker-Heights
East-Elmhurst
East-Flatbush
East-Harlem
East-New-York
East-Tremont
East-Village
Eastchester
Elm-Park
Elmhurst
Eltingville
Emerson-Hill
Far-Rockaway
Fieldston
Financial-Di

Now, we will apply this function to the rest of cities. Once extracted the data, we will go ahead and analyze the neighborhoods with highest and lowest income. Pandas has a useful function called "sort_values" which orders the rows of a Dataframe in accordance to a particular column values. In our case, the ordering of rows will be in accordance to the "Income" column. 

In [576]:
neigh_ws_2 = pd.DataFrame(city_neigh_extractor('District-of-Columbia', 'Washington'), 
                           columns=['City', 'Neighborhood', 'Population', 'Household Income']) 

neigh_hs_2 = pd.DataFrame(city_neigh_extractor('Texas', 'Houston'), 
                          columns=['City', 'Neighborhood', 'Population', 'Household Income'])

neigh_la_2 = pd.DataFrame(city_neigh_extractor('California', 'Los-Angeles'),
                          columns=['City', 'Neighborhood', 'Population', 'Household Income'])

In [636]:
# More rows and columns to visualize
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 5000)

list_of_neigh2 = [neigh_ny_2, neigh_ws_2, neigh_hs_2, neigh_la_2]
#display(neigh_ny_2)

ngh_cities = pd.DataFrame()
for ngh in list_of_neigh2:
    ngh_sort = ngh[~ngh['Household Income'].str.contains("%")]
    ngh_sort['Population'] = pd.to_numeric(ngh_sort['Population'], downcast="integer")
    ngh_sort['Household Income'] = pd.to_numeric(ngh_sort['Household Income'], downcast="float").round(0).astype('int')
    ngh_sort = ngh_sort.sort_values(by=['Household Income'])
    
    print('{}\'s neighborhoods with lowest median household income:'.format(ngh['City'][0]))
    display(ngh_sort.head())
    print('{}\'s neighborhoods with highest median household income:'.format(ngh['City'][0]))
    display(ngh_sort.tail())
    print(' ')
    
    ngh_df = pd.concat([ngh_sort.head(3), ngh_sort.tail(3)])
    ngh_cities = ngh_cities.append(ngh_df)
    

New-York's neighborhoods with lowest median household income:


C:\Users\adminit\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\adminit\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,City,Neighborhood,Population,Household Income
237,New-York,West-Farms,18053,22652
151,New-York,Morris-Heights,40982,23172
153,New-York,Mott-Haven,51381,23195
135,New-York,Longwood,39965,23677
70,New-York,East-Tremont,33593,24395


New-York's neighborhoods with highest median household income:


,City,Neighborhood,Population,Household Income
218,New-York,Sutton-Place,21830,154995
8,New-York,Battery-Park,13386,164676
37,New-York,Carnegie-Hill,15600,179100
224,New-York,Tribeca,19794,193906
61,New-York,DUMBO,3033,225120


...

,City,Neighborhood,Population,Household Income
53,Houston,Memorial,46572,119809
14,Houston,Downtown,16716,120242
80,Houston,Washington-Avenue-Coalition---Memorial-Park,26574,126958
79,Houston,University-Place,15811,129745
2,Houston,Afton-Oaks---River-Oaks-Area,13718,159915


 
Los-Angeles's neighborhoods with lowest median household income:


,City,Neighborhood,Population,Household Income
82,Los-Angeles,University-Park,25100,20716
61,Los-Angeles,Pico-Union,28751,25615
15,Los-Angeles,Chinatown,7798,26993
87,Los-Angeles,Watts,39593,29606
25,Los-Angeles,Elysian-Park,825,29692


Los-Angeles's neighborhoods with highest median household income:


,City,Neighborhood,Population,Household Income
14,Los-Angeles,Cheviot-Hills,13062,135290
8,Los-Angeles,Brentwood,32776,152862
5,Los-Angeles,Beverly-Glen,13858,157467
56,Los-Angeles,Pacific-Palisades,24798,181003
4,Los-Angeles,Bel-Air,8033,188097


In [637]:
ngh_cities = ngh_cities.reset_index(drop=True)

# Change the hyphen values, '-', into a space, ' ', as we will use the neighborhood names as input to extract 
# the neighborhood polygons. 
ngh_cities['City'] = ngh_cities['City'].str.replace('-', ' ')
ngh_cities['Neighborhood'] = ngh_cities['Neighborhood'].str.replace('-', ' ')

# Change "Washington" to "Washington D.C." so that we could further use this string to extract the random points inside the 
# polygons of a particular neighborhood
ngh_cities['City'] = ngh_cities['City'].replace('Washington', 'Washington D.C.')

display(ngh_cities)

,City,Neighborhood,Population,Household Income
0,New York,West Farms,18053,22652
1,New York,Morris Heights,40982,23172
2,New York,Mott Haven,51381,23195
3,New York,Carnegie Hill,15600,179100
4,New York,Tribeca,19794,193906
5,New York,DUMBO,3033,225120
6,Washington D.C.,Barry Farm,2307,14563
7,Washington D.C.,Ivy City,858,17303
8,Washington D.C.,Gallaudet,1179,17303
9,Washington D.C.,Hawthorne,754,210357


In [608]:
dict_maps3 = random_point_generator(ngh_cities['City'], ngh_cities['Neighborhood'], num_points=5, 
                                    neigh_format=True, list_of_neigh=ngh_cities['Neighborhood'])


dict_maps3 = save_dict_maps('dict_maps3', dict_maps3)
print(dict_maps3.keys())

Done: West Farms
Done: Morris Heights
Done: Mott Haven
Done: Carnegie Hill
Done: Tribeca
Done: DUMBO
Done: Barry Farm
Done: Ivy City
Done: Gallaudet
Done: Hawthorne
Done: Spring Valley
Done: Massachusetts Heights
Done: Kashmere Gardens
Done: Westwood
Done: University Place
Done: University Park
Done: Pico Union
Done: Chinatown
Done: Beverly Glen
Done: Pacific Palisades
Done: Bel Air
dict_keys(['West Farms', 'Morris Heights', 'Mott Haven', 'Carnegie Hill', 'Tribeca', 'DUMBO', 'Barry Farm', 'Ivy City', 'Gallaudet', 'Hawthorne', 'Spring Valley', 'Massachusetts Heights', 'Kashmere Gardens', 'Westwood', 'University Place', 'University Park', 'Pico Union', 'Chinatown', 'Beverly Glen', 'Pacific Palisades', 'Bel Air'])


In [ ]:
neigh_venues = venue_extractor(dict_maps3, neigh_format=True, neigh_data=ngh_cities)

neigh_venues = save_dict_maps('neigh_venues', neigh_venues)

In [754]:
whole_list_2 = []

for n in neigh_venues:
    for ip in n:
        whole_list_2.append(ip)
        
        
whole_neigh_df = pd.DataFrame(whole_list_2, columns=['City', 'Neighborhood', 'Population', 
                                                     'Household Income', 'Bank Establishment', 
                                                     'Latitude', 'Longitude'])


whole_neigh_df = whole_neigh_df.drop_duplicates()
display(whole_neigh_df)

,City,Neighborhood,Population,Household Income,Bank Establishment,Latitude,Longitude
0,New York,West Farms,18053,22652,Chase Bank,40.828438,-73.878337
1,New York,West Farms,18053,22652,Chase Bank,40.837704,-73.882952
2,New York,West Farms,18053,22652,TD Bank,40.857772,-73.884187
3,New York,West Farms,18053,22652,Chase Bank,40.855670,-73.887058
4,New York,West Farms,18053,22652,Chase Bank,40.854090,-73.867347
...,...,...,...,...,...,...,...
8098,Los Angeles,Bel Air,8033,188097,Union Bank,34.066917,-118.399309
8099,Los Angeles,Bel Air,8033,188097,1st Century Bank,34.067265,-118.405583
8100,Los Angeles,Bel Air,8033,188097,Pacific Premier Bank,34.042365,-118.468136
8101,Los Angeles,Bel Air,8033,188097,Money Mart,34.040694,-118.464015


In [813]:
# Function to calculate the ratio between the number of bank establishments and population for each neighborhood
def divide_two_cols(df_sub):
    return float(df_sub['Bank Establishment'].count())/ (df_sub['Population'].mode()/1000) 


# New York 
ny_df_2 = whole_neigh_df[whole_neigh_df['City'] == 'New York']

ny_df_2 = ny_df_2.groupby(['Neighborhood', 'Household Income']).apply(
          divide_two_cols).reset_index().sort_values(by='Household Income')

ny_df_2 =  ny_df_2.rename(columns={0: 'Bank Establishments/Population (per Thousand)'})

display(ny_df_2)


# Washington
wa_df_2 = whole_neigh_df[whole_neigh_df['City'] == 'Washington D.C.']

wa_df_2 = wa_df_2.groupby(['Neighborhood', 'Household Income']).apply(
          divide_two_cols).reset_index().sort_values(by='Household Income')

wa_df_2 =  wa_df_2.rename(columns={0: 'Bank Establishments/Population (per Thousand)'})

display(wa_df_2)


# Houston

hs_df_2 = whole_neigh_df[whole_neigh_df['City'] == 'Houston']

hs_df_2 = hs_df_2.groupby(['Neighborhood', 'Household Income']).apply(
          divide_two_cols).reset_index().sort_values(by='Household Income')

hs_df_2 =  hs_df_2.rename(columns={0: 'Bank Establishments/Population (per Thousand)'})

display(hs_df_2)


# Los Angeles

la_df_2 = whole_neigh_df[whole_neigh_df['City'] == 'Los Angeles']

la_df_2 = la_df_2.groupby(['Neighborhood', 'Household Income']).apply(
          divide_two_cols).reset_index().sort_values(by='Household Income')

la_df_2 =  la_df_2.rename(columns={0: 'Bank Establishments/Population (per Thousand)'})

display(la_df_2)

,Neighborhood,Household Income,Bank Establishments/Population (per Thousand)
5,West Farms,22652,5.539246
2,Morris Heights,23172,2.440096
3,Mott Haven,23195,1.946245
0,Carnegie Hill,179100,8.205128
4,Tribeca,193906,6.214004
1,DUMBO,225120,32.970656


,Neighborhood,Household Income,Bank Establishments/Population (per Thousand)
0,Barry Farm,14563,48.981361
1,Gallaudet,17303,84.817642
3,Ivy City,17303,116.550117
2,Hawthorne,210357,139.257294
5,Spring Valley,242717,23.916562
4,Massachusetts Heights,250001,568.306011


,Neighborhood,Household Income,Bank Establishments/Population (per Thousand)
0,Kashmere Gardens,23893,0.629789
2,Westwood,25494,3.069302
1,University Place,129745,2.213649


,Neighborhood,Household Income,Bank Establishments/Population (per Thousand)
5,University Park,20716,5.338645
4,Pico Union,25615,5.565024
2,Chinatown,26993,12.823801
1,Beverly Glen,157467,1.226728
3,Pacific Palisades,181003,0.120977
0,Bel Air,188097,12.448649
